## **교보문고 도서검색**


* **학습 목표**
    * 검색명 / 페이지 수 / 정렬기준에 따라 크롤링 할 수 있도록 함수를 만들 수 있다.
    * 카테고리, 도서분류, 도서명, 설명, 저자, 출판사, 출판일, 가격, 평점, 리뷰 수가 나타나는 데이터프레임을 만들 수 있다.

    ※ <u>각 상품마다 모든 정보가 무조건 있는 것은 아니다. 없는 것도 있기 때문에 예외 처리를 어떻게 해야할 지 고민해야 한다.</u><br>
    ※ <u>도서와 강연이 모두 검색되어 검색된 강연은 제외하였다.</u><br>
    ※ ★★주의★★<br><b><u>class name에 너무 의지하면 안된다.(class name이 겹치는 경우가 많기 때문에 `find`나 `select_one`을 이용할 때 원하는 정보를 가져오는데 실수할 수 있다.)</u></b>
<div markdown="1" style="text-align:center; margin-bottom:10px">
<img src="./images/kyobo_html.PNG" width="50%">
</div> 

## **1) URL 파악하기**

* `https://search.kyobobook.co.kr/search?조건들`
* 조건들: 여러 조건일 때 '&'로 구분한다.

    <table>
        <tr> <th>key</th> <th>설명</th> </tr>
        <tr> <td>keyword</td> <td>검색할 도서 명</td>
        <tr> <td>page</td> <td>페이지 번호</td></tr>
        <tr> 
            <td>ra(쓰지 않으면 default 인기순)</td> 
            <td> 
                - qntt : 판매량순<br>
                - date : 최신순<br>
                - name : 상품명순<br>
                - dc : 할인율순<br>
                - prasc : 낮은가격순<br>
                - prdesc : 높은가격순<br>
                - kcont : 클로버리뷰순<br>
                - krvgr : 클로버평점순
            </td> 
        </tr>  
    </table>


## **2) HTML 파악하기**

* li, class='prod_item'
    * div, class='auto_overflow_wrap prod_name_group'
        * a, class='prod_info'
            * span, class='prod_category'의 text(0번째)$\rightarrow$ `category(도서분류)`
            * span, id(마지막) $\rightarrow$ `book_name(도서명)`
    * div, class='prod_desc_info'
        * span, class='prod_desc'의 text $\rightarrow$ `description(설명)`
    * div, class='prod_author_info'
        * a, class='author rep'의 text (여러 개 있을 수 있다) $\rightarrow$ `authors(저자)`
        * div, class='prod_publish'
            * a, class='text'의 text $\rightarrow$ `publisher(출판사)`
            * span, class='date'의 text $\rightarrow$ `publish_date(출판일)`
    * span, class='price'
        * span, class='val'의 text $\rightarrow$ `price(가격)`
    * div, class='prod_bottom'
        * span, review_klober_box
            * 0번째의 text $\rightarrow$ `points(평점)`
            * 1번째의 text $\rightarrow$ `reviews(리뷰 수)`

## **3) 함수 만들기**

In [89]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import re 

# Function
def GyoboMungo(keyword, page_no, option):
    options = {0: ('','인기순'), 1: ('qntt', '판매량순'), 2: ('date', '최신순'), 3: ('name','상품명순'), 
            4: ('dc', '할인율순'), 5: ('prasc', '낮은가격순'), 6: ('prdesc', '높은가격순'), 
            7: ('kcont', '클로버리뷰순'), 8: ('krvgr', '클로버평점순')}
    columns = ['검색명', '카테고리', '도서명', '요약', '저자', '출판사', '출판일', '가격', '평점', '리뷰 수']

    # Crawling
    print(f'{keyword}와 관련된 도서를 {options[option][1]}으로 검색합니다.')
    print('Start>', end='')
    book_lst = []
    cnt = 0

    ## Each Page
    for p in range(page_no):
        # print message(each page)
        print(f'page {p+1}', end='')

        # HTML 추출
        url = f'https://search.kyobobook.co.kr/search?keyword={keyword}&p={p+1}&ra={option}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

        response = req.get(url, headers=headers)
        html = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        books = soup.select('div.switch_prod_wrap.view_type_list > ul.prod_list > li.prod_item')

        # Each Item
        for i, book in enumerate(books):
            # 도서분류 / 도서명
            book_html = book.select('a.prod_info > span')
            
            category = book_html[0].text
            if category == '[강연]':
                continue
            else:
                category = re.search('(?<=\[)(.*?)(?=\])',category).group()
            book_name = book_html[-1].text

            # 도서 설명
            description = book.select('span.prod_desc')[-1].text

            # 저자 
            author_css = 'div.auto_overflow_wrap.prod_author_group > div.auto_overflow_contents > div.auto_overflow_inner > '
            au_html = book.select(author_css + 'a')
            au_type_html = book.select(author_css + 'span.type')

            authors = {}
            for au, au_type in zip(au_html, au_type_html):
                if au_type.text in authors.keys():
                    authors[au_type.text].append(au.text) 
                else:
                    authors[au_type.text] = [au.text]
        
            # 출판사 / 출판일
            publish_css = 'div.prod_publish > '

            publisher = book.select_one(publish_css + 'a.text').text
            publish_date = book.select_one(publish_css + 'span.date').text

            # 가격    
            price = int(re.sub('[^0-9]','',book.select_one('span.price > span.val').text))
            
            # 평점 / 리뷰 수
            klover_html = book.select('span.review_klover_box > span')

            points = float(klover_html[0].text)
            reviews = int(re.sub('[^0-9]','',klover_html[1].text))

            # 행 추가
            cnt += 1
            book_lst.append([keyword, category, book_name, description, authors, publisher, publish_date, price, points, reviews])

            # print message(each item)
            if (i + 1) % 5 == 0:
                print('>', end='')
            
    print('End')
    print(f'{keyword}와 관련된 도서 {cnt}개의 정보를 수집했습니다')

    book_df = pd.DataFrame(book_lst, columns=columns)
    
    return book_df

In [90]:
# Settings
keyword = '인공지능'
page_no = 3
option = 0

kyobo_df = GyoboMungo(keyword, page_no, option)
kyobo_df


인공지능와 관련된 도서를 인기순으로 검색합니다.
Start>page 1>>>>page 2>>>>page 3>>>>End
인공지능와 관련된 도서 54개의 정보를 수집했습니다


,검색명,카테고리,도서명,요약,저자,출판사,출판일,가격,평점,리뷰 수
0,인공지능,국내도서,"빅데이터, 인공지능을 만나다",,"{'저자(글)': ['심준식', '우재현']}",한국금융연수원,2023년 04월 03일,17100,0.00,0
1,인공지능,국내도서,대화형 인공지능 천재가 되다,공부가 재미있어지는 챗gpt 활용법,"{'저자(글)': ['빅아이 인공지능 연구소'], '감수': ['조상']}",Oldstairs(올드스테어즈),2023년 04월 01일,13500,10.00,4
2,인공지능,국내도서,인공지능 교과서 2,,"{'감수': ['AI PLUS 피지컬컴퓨팅교사연구회'], '책임편집': ['천위쿤'...",광문각,2020년 11월 30일,14400,0.00,0
3,인공지능,국내도서,인공지능윤리 다원적 접근,,{'저자(글)': ['인공지능과 가치 연구회']},박영사,2021년 02월 10일,14000,10.00,3
4,인공지능,국내도서,인공지능 교과서 1,,"{'책임편집': ['천위쿤'], '감수': ['AI PLUS 피지컬컴퓨팅교사연구회'...",광문각,2020년 07월 15일,16000,0.00,0
5,인공지능,국내도서,챗GPT-4 인공지능 미래세상,챗GPT와 GPT-4 인공지능이 세상을 바꾸고 있다.,{'저자(글)': ['안종배']},광문각출판미디어,2023년 04월 12일,25200,10.00,4
6,인공지능,국내도서,모두의 인공지능 기초 수학,(모두의 시리즈),{'저자(글)': ['서지영']},길벗,2020년 08월 06일,22500,8.90,31
7,인공지능,국내도서,누구나 쉽게 따라하는 인공지능(feat 파이썬),핵심만 쏙쏙 담은 생활밀착형 인공지능 입문서,{'저자(글)': ['이용권']},씨마스21,2021년 08월 25일,18900,8.44,11
8,인공지능,국내도서,모두가 할 수 있는 앱 인벤터로 11개 인공지능 앱 만들기,초중고 대학생 모두를 위한 블록 코딩으로 인공지능 머신러닝 이해하기!,"{'저자(글)': ['박지숙', '김수연', '전진아', '김종렬', '장문철']}",앤써북,2021년 06월 30일,13050,10.00,11
9,인공지능,국내도서,인공 지능 음악,인공 지능 소개 및 스크래치와 파이썬 활용,{'저자(글)': ['최영준']},예솔,2023년 03월 02일,25200,0.00,0
